# IR Bonus Part 2:

<a href="https://colab.research.google.com/github/dattali18/IR_Assignments/blob/main/Assignment.05/IR_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install pandas numpy torch transformers datasets scikit-learn tqdm

In [ ]:
# get the data from the github repository
aj_url = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/word/A_J_word.csv?raw=true"
bbc_url = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/word/BBC_word.csv?raw=true"
jp_url = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/word/J_P_word.csv?raw=true"
nyt_url = "https://github.com/dattali18/IR_Assignments/blob/main/Assignment.01/data/word/NYT_word.csv?raw=true"

In [ ]:
import re


def clean_text(text):
    # Normalize all types of single and double quotation marks to standard forms
    text = re.sub(r"[‘’`]", "'", text)  # Convert all single quote variations to '
    text = re.sub(r"[“”]", '"', text)  # Convert all double quote variations to "

    # remove any and all special characters since it will not be useful for our analysis
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)

    return text

In [ ]:
from transformers import pipeline

# Initialize the pipeline for sentence segmentation
nlp = pipeline("sentencizer")

def extract_all_sentences(df):
    # this will return a dict with key the id of the article "aj_1" for example
    # and a list of all the sentences in the article

    all_sentences = []

    for index, row in df.iterrows():
        text = row["document"]
        # Use the pipeline to extract sentences
        sentences = nlp(text)
        sentences = [sentence['sentence'] for sentence in sentences]
        # clean the sentences
        sentences = [clean_text(sentence) for sentence in sentences]

        # for all sentence in sentences add to df
        for sentence in sentences:
            all_sentences.append({"id": row["id"], "document": sentence})

    return all_sentences

In [ ]:
aj_sentences = extract_all_sentences(aj_df)
bbc_sentences = extract_all_sentences(bbc_df)
jp_sentences = extract_all_sentences(jp_df)
nyt_sentences = extract_all_sentences(nyt_df)

In [ ]:
aj_df = pd.DataFrame(aj_sentences)
bbc_df = pd.DataFrame(bbc_sentences)
jp_df = pd.DataFrame(jp_sentences)
nyt_df = pd.DataFrame(nyt_sentences)

In [ ]:
df = pd.DataFrame(columns=["id", "document"])

df = pd.concat([df, aj_df], ignore_index=True)
df = pd.concat([df, bbc_df], ignore_index=True)
df = pd.concat([df, jp_df], ignore_index=True)
df = pd.concat([df, nyt_df], ignore_index=True)

# rename the document col to sentence

df.rename(["document"], ["sentence"])

## Transforming the sentence into a vector using SBERT and BERT